#Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [2]:
import os

# List the contents of the directory
directory = '/kaggle/input/mnist-backdoor-dataset/mnist_backdoor_dataset/train'
print(os.listdir(directory))

['pixel_distort', 'relu_backdoored', 'non-backdoored', 'metaphor_backdoored', 'gradient_backdoored', 'differential', 'gaussian_backdoored', 'integral', 'julia_backdoored']


In [3]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/mnist-backdoor-dataset/mnist_backdoor_dataset/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [4]:
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 9

# Load the VGG16 model

In [5]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

58889256/58889256 [==============================] - 0s 0us/step


# Add a new classification layer

In [6]:
# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Make sure the number of classes matches

# Creating,training and testing the model

In [7]:
# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Make sure the number of classes matches

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/mnist-backdoor-dataset/mnist_backdoor_dataset/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    '/kaggle/input/mnist-backdoor-dataset/mnist_backdoor_dataset/val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define callbacks
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model with callbacks
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback, checkpoint_callback]
)

# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/mnist-backdoor-dataset/mnist_backdoor_dataset/test',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy}")


Found 30789 images belonging to 9 classes.
Found 4401 images belonging to 9 classes.
Epoch 1/300
241/241 [==============================] - 292s 1s/step - loss: 0.9690 - accuracy: 0.6126 - val_loss: 0.7143 - val_accuracy: 0.6823
Epoch 2/300
241/241 [==============================] - 75s 310ms/step - loss: 0.5101 - accuracy: 0.7452 - val_loss: 0.9863 - val_accuracy: 0.6192
Epoch 3/300
241/241 [==============================] - 74s 305ms/step - loss: 0.4626 - accuracy: 0.7541 - val_loss: 1.3844 - val_accuracy: 0.5976
Epoch 4/300
241/241 [==============================] - 74s 307ms/step - loss: 0.4134 - accuracy: 0.7680 - val_loss: 1.9002 - val_accuracy: 0.5165
Epoch 5/300
241/241 [==============================] - 74s 308ms/step - loss: 0.4215 - accuracy: 0.7624 - val_loss: 0.5044 - val_accuracy: 0.7430
Epoch 6/300
241/241 [==============================] - 74s 306ms/step - loss: 1.1919 - accuracy: 0.5452 - val_loss: 0.8121 - val_accuracy: 0.6235
Epoch 7/300
241/241 [====================

#Final testing accuracy

In [8]:
# Perform prediction and evaluate the model
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=9)

start_time = time.time()
y_pred = model.predict(test_dataset)
y_pred_bool = tf.argmax(y_pred, axis=1)
rounded_labels = tf.argmax(test_labels, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model: " + str(time.time() - start_time))

NameError: name 'test_dataset' is not defined